upsert using Merge setatement:

1. insert data
2. update data
3. delete data

all 3 in one statement


Day 1 freash data.

In [0]:
# drivers_day1_df = spark.read.\
#     .option("inferSchema", True) \
#     .json('abfss://raw@dlformulaone2024.dfs.core.windows.net/2021-03-28/drivers.json')\
#     .filter("driverId <= 10")\
#    .select("driverID", "dob", "name.forename", "name.surname")

drivers_day1_df = spark.read\
    .option("inferSchema", True)\
    .json('abfss://raw@dlformulaone2024.dfs.core.windows.net/2021-03-28/drivers.json')\
    .filter("driverId <= 10")\
    .select("driverID", "dob", "name.forename", "name.surname")

In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")

Day 2 data. got data from id 6 to 15. and from 6 to 10 with updated data.

In [0]:
from pyspark.sql.functions import upper

drivers_day2_df = spark.read\
    .option("inferSchema", True)\
    .json('abfss://raw@dlformulaone2024.dfs.core.windows.net/2021-03-28/drivers.json')\
    .filter("driverId between 6 and 15")\
    .select("driverID", "dob", upper("name.forename").alias('forename'), upper("name.surname").alias("surname"))

In [0]:
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
drivers_day3_df = spark.read\
    .option("inferSchema", True)\
    .json('abfss://raw@dlformulaone2024.dfs.core.windows.net/2021-03-28/drivers.json')\
    .filter("driverId between 1 and 5 or driverId between 16 and 20")\
    .select("driverID", "dob", upper("name.forename").alias('forename'), upper("name.surname").alias("surname"))

In [0]:
%sql
create table if not exists f1_delta.drivers_merge(
  driverId int,
  dob date,
  forename string,
  surname string,
  createDate date,
  updateDate date
)
using delta

In [0]:
%sql
merge into f1_delta.drivers_merge tgt
using drivers_day1 upd
on tgt.driverId = upd.driverId
when matched then
  update set tgt.dob = upd.dob,
            tgt.forename = upd.forename,
            tgt.surname = upd.surname,
            tgt.updateDate = current_timestamp
when not matched 
  then insert (driverId, dob, forename, surname, createDate) values (driverId, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
merge into f1_delta.drivers_merge tgt
using drivers_day2 upd
on tgt.driverId = upd.driverId
when matched then
  update set tgt.dob = upd.dob,
            tgt.forename = upd.forename,
            tgt.surname = upd.surname,
            tgt.updateDate = current_timestamp
when not matched 
  then insert (driverId, dob, forename, surname, createDate) values (driverId, dob, forename, surname, current_timestamp)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,5,0,5


In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, 'abfss://demo@dlformulaone2024.dfs.core.windows.net/drivers_merge')

deltaTable.alias("tgt").merge(
    drivers_day3_df.alias("upd"),
    "tgt.driverId = upd.driverId")\
    .whenMatchedUpdate(set = {"dob" :"upd.dob", 
                              "forename" : "upd.forename", 
                              "surname" : "upd.surname",
                              "updateDate" : "current_timestamp()"})\
    .whenNotMatchedInsert(values=
                          {
                            "driverId" : "upd.driverId",
                            "dob" : "upd.dob",
                            "forename" : "upd.forename",
                            "surname" : "upd.surname",
                            "createDate" : "current_timestamp()"
                          }
                        )\
    .execute()
    

In [0]:
%sql
select * from f1_delta.drivers_merge

driverId,dob,forename,surname,createDate,updateDate
6,1985-01-11,KAZUKI,NAKAJIMA,2024-02-23,2024-02-23
7,1979-02-28,SÉBASTIEN,BOURDAIS,2024-02-23,2024-02-23
8,1979-10-17,KIMI,RÄIKKÖNEN,2024-02-23,2024-02-23
9,1984-12-07,ROBERT,KUBICA,2024-02-23,2024-02-23
10,1982-03-18,TIMO,GLOCK,2024-02-23,2024-02-23
11,1977-01-28,TAKUMA,SATO,2024-02-23,null
12,1985-07-25,NELSON,PIQUET JR.,2024-02-23,null
13,1981-04-25,FELIPE,MASSA,2024-02-23,null
14,1971-03-27,DAVID,COULTHARD,2024-02-23,null
15,1974-07-13,JARNO,TRULLI,2024-02-23,null
